# HealthGrader List

In [ ]:
url_list = ['https://www.healthgrades.com/physician/dr-lisa-kairis-xt5b8',
 'https://www.healthgrades.com/physician/dr-sophie-hoptowit-2dwt6',
 'https://www.healthgrades.com/physician/dr-stephen-kelminson-x76s8',
 'https://www.healthgrades.com/physician/dr-jonathan-weeks-x3v44',
 'https://www.healthgrades.com/physician/dr-walter-guarino-2qfmv',
 'https://www.healthgrades.com/providers/ella-gray-3mpy5',
 'https://www.healthgrades.com/physician/dr-john-condojani-yrrkl',
 'https://www.healthgrades.com/physician/dr-anthony-leazzo-yjqx4',
 'https://www.healthgrades.com/physician/dr-james-gleason-yt27c',
 'https://www.healthgrades.com/physician/dr-katherine-francis-x2cw7']


# Focus on One

In [ ]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
import re

import requests
from scrapy.http import TextResponse


# we need headers to disguise our bot as a browser

headers = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
    "Accept-Encoding": "gzip,deflate,sdch",
    "Accept-Language": "zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-TW;q=0.2",
}


In [ ]:
# import cloudscraper
# scraper = cloudscraper.create_scraper()

# url = 'https://www.healthgrades.com/physician/dr-ravi-bajaj-2pxm9'
# url = 'https://www.healthgrades.com/physician/dr-joseph-nichols-gfpd5' # 32 rating

# url = 'https://www.healthgrades.com/providers/ella-gray-3mpy5' # 100+ ratings
url = 'https://www.healthgrades.com/physician/dr-walter-guarino-2qfmv' # 
# as we cannot use Download Middleware in Scrapy, requests are used here instead.
r = requests.get(url, headers = headers)
r.url

In [ ]:

# load the text to scrapy-type response
response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

# have a look at its body
print(str(response.body.decode()))

with open('_tmp.html', 'w') as f:
    f.write(str(response.body.decode()))


# All Data

Get information from javascript

In [ ]:
import json

def process_Json(response):
    xpath = './/script[@type="text/javascript"]//text()'
    selectors = response.xpath(xpath)
    js_data = [i for i in selectors.extract()[3].split('\n')  if '= {' in i]
    js_data = {t[0].strip(): json.loads(t[1][:-1]) for t in [i.split(' = ') for i in js_data]}
    return js_data


js_data = process_Json(response)
[i for i in js_data]

# Pes

In [ ]:
def process_Pes(Pes):
    d = {}
    Pes_Model = Pes['model']
    overall = Pes_Model['overall']
    for item in ['responseCount','reviewCount','actualScore','roundedScore',]:
        d[item] = overall[item]
        
    d['score_aggregates'] = Pes_Model["surveyDistribution"]['aggregates']
    d['lastSurveyDate'] = Pes_Model['lastSurveyDate']
    d['doc_pfmc'], d['staff_pfmc'] = [i['aggregates'] for i in Pes_Model['cards']]
    # at most 20 reviews
    d['comments'] = Pes['model']['comments']['results']
    return d


Pes = js_data['pageState.pes']

d = process_Pes(Pes)
# print(d['reviewCount'])
# len(d['comments'])

d['reviewCount'] > len(d['comments'])

In [ ]:
# Pes = js_data['pageState.pes']
# print([i for i in Pes])
# for item in [ 'commentsSuppressed', 'hasComments', 'serverError']:
#     print('\n\n' + item, Pes[item])


# print([i for i in Pes['model']])
# Pes_Model = Pes['model']



# cols = [ 'insights',  'recaptchaSitekey', 'launchDarklyKey', 'displayName',
#         'providerCode', 'providerName', 'providerPossessiveName', 'providerUrl', 
#         'surveysSuppressed', 'commentsSuppressed', 'lastSurveyDate',
#         'flagReasons', 'newTemplateModel', 
#         'useNewForm', 'providerResponseAboutMe', 
#         'smallImageUrl', 'mediumImageUrl', 'largeImageUrl', 
#         'sponsorshipImageUrl', 'specialty', 'hideProviderCTA', 
#         'commentCount', 'verifiedPatientSurveys', 'renderCombinedComments']


# for i in cols:
#     print('\n\n'+i)
#     print(Pes_Model[i])

# pprint(Pes_Model["providerResponseAboutMe"])

In [ ]:
# d = Pes_Model['overall']
# pprint(d)

# pprint(Pes_Model["surveyDistribution"])

# to ask the user to answer the question about the focal doctor
# pprint(Pes_Model["templateModel"])


# Pes_Model['lastSurveyDate']


# doc_pfmc, staff_pfmc = Pes_Model['cards']
# pprint(doc_pfmc)
# pprint(staff_pfmc)

# viewModel

In [ ]:
def process_viewModel(viewModel):
    cols = ['npi', 'pwid', 'entityType', 'websiteUrl', 'logoUrl', 'imageUrl', 
     'providerUrl', 'displayName', 'displayNamePossessive', 'providerDisplayFullName', 
     'age', 'genderString', 'badges', 'languages', 
     'hasAutoBiography', 'aboutMe', 'aboutMeVideoUrl', 
     'aboutProvider', 'description', 'generatedbiography', 'autoBiography', 'cityNameAndState', 
     'officeLocations', 'officePhone', 'acceptsNewPatients', 
     'availability', 
     'practicingSpecialties', 'medicalSpecialty', 
     'awardsAndRecognitions', 'boardCertifications', 'insuranceAccepted', 
     'isPrimaryLocationMalpracticeCollected', 'malpractices', 
     'sponsorName', 'boardActions', 'memberships', 
     'education', 'hospitals', 'hasConditions', 
     'hasProcedures',  'existingPatientPhone', 
     'readStoryScrolloffset', 'sanctions', 
     'shouldShowVideoContent', 'showPatientVolumeData', 'showVisitingSection', 
     'specialtyHasClinicalFocus', 'specialtyHeaderText', 'suppressCertifications',
     'suppressSurveys', 'syndication', 'testimonies', 'uconnectEnvironment', 'writeMd',
     'conditionsAndProcedures', 'clinicalFocusItems',]
    d = {}
    for i in cols:
        d[i] = viewModel[i]
    return d


viewModel = js_data['pageState.viewModel']

d = process_viewModel(viewModel)
d

In [ ]:
# viewModel = js_data['pageState.viewModel']
# print([i for i in viewModel])
# pprint(viewModel['model'])
# selected_items = []

## Basic Info

In [ ]:
# info = [ 
# 'npi', 'pwid', 'entityType', 
# 'websiteUrl', 'logoUrl', 'imageUrl', 'providerUrl', 
# 'displayName','displayNamePossessive', 'providerDisplayFullName',
# 'age',  'genderString', # 'gender',
# 'badges', 'languages',
# 'hasAutoBiography', 
# 'aboutMe', 'aboutMeVideoUrl', 'aboutProvider', 
# 'description', 'generatedbiography',  'autoBiography', 
# ]


# selected_items += [i for i in info if i not in selected_items]

# for item in info:
#     print('\n\n' + item)
#     print(viewModel[item])

## Location Info

In [ ]:
# info = [
# 'cityNameAndState',
# 'officeLocations',
# 'officePhone',
# # 'primaryOffice', 
# # 'primaryOfficeRegionName',   
# ]


# selected_items += [i for i in info if i not in selected_items]



# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Rating and Review

In [ ]:
# we don't need this
# info = ['displayOverallStarRating','reviewItemProp',  'surveyUserCount',  ]

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## clinical and practicing

In [ ]:
# info = [
# # 'primaryPracticingSpecialty', 
# 'acceptsNewPatients',
# 'availability', 
# # 'practicingSpecialityName', 
# 'practicingSpecialties', 
# # 'practicingSpecialtyNameIst', 
# # 'practicingSpecialtyNameIsts', 
# # 'practicingSpecialtyNamePlural',
# 'medicalSpecialty']


# selected_items += [i for i in info if i not in selected_items]

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## award info

In [ ]:
# info = [
# 'awardsAndRecognitions', 
# 'boardCertifications', 
# # 'boardCertificationFootnotes',  
# ]

# selected_items += [i for i in info if i not in selected_items]

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Insurance

In [ ]:
# info = [
# 'insuranceAccepted', 
# # 'insurancePhone',  
# ]


# selected_items += [i for i in info if i not in selected_items]


# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Mal Info

In [ ]:
# # mal practice
# info = [
# 'isPrimaryLocationMalpracticeCollected', 
# 'malpractices',  ]

# selected_items += [i for i in info if i not in selected_items]

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Sponsor Info

In [ ]:

# info = [ 
# 'sponsorName', 
# # 'sponsoringFacilityAward', 
# # 'sponsoringFacilityLink',
# # 'sponsoringFacilityName', 
# ]


# selected_items += [i for i in info if i not in selected_items]


# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Board, Membership

In [ ]:
# info = [ 'boardActions', 'memberships', ]


# selected_items += [i for i in info if i not in selected_items]


# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Education

In [ ]:


# info = [  'education', ]


# selected_items += [i for i in info if i not in selected_items]



# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Hospitals

In [ ]:
# info = [ 'hospitals',  ]

# selected_items += [i for i in info if i not in selected_items]

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Conditions&Procedures

In [ ]:
# # not sure what they are, but keep them
# info = [ 
#     'hasConditions', 'hasProcedures', 'conditionsAndProcedures', 'clinicalFocusItems' 
# ]

# selected_items += [i for i in info if i not in selected_items]

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])

## Patients

In [ ]:

# info = ['existingPatientPhone']


# selected_items += [i for i in info if i not in selected_items]


# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])
    
    

## Others

In [ ]:
# info = [
# 'readStoryScrolloffset', 
# 'sanctions', 
# 'shouldShowVideoContent', 
# 'showPatientVolumeData', 'showVisitingSection', 
# 'specialtyHasClinicalFocus', 
# 'specialtyHeaderText', 
# 'suppressCertifications', 
# 'suppressSurveys', 
# 'syndication', 
# 'testimonies', 
# 'uconnectEnvironment', 
# 'writeMd', 
# ]



# selected_items += [i for i in info if i not in selected_items]



# # print(newitem)

# for item in info:
#     print('\n\n' + item)
#     pprint(viewModel[item])
    
    
    
    

In [ ]:
# selected_items

# Xpath Data

In [ ]:
def process_xpath(response):


    item2xpath = {
        'biography': './/*[@data-qa-target="premium-biography"]//text()',
        # 'strengths_item': './/li[@class="provider-strengths-item"]',
        # 'rate_score': './/p[@class="score"]//text()',
        # 'rate_num': './/p[@class="survey-count"]//text()',
        # 'rate_details':'.//table[@class="rating-table"]//text()', 
    }



    itemselector2value = {
        'biography': lambda x: x.extract(),
        #  'strengths_item': lambda x: [''.join(i.xpath('.//div//text()').extract()[:4]) for i in x],
        # 'rate_score': lambda x: float(x.extract()[0]), 
        # 'rate_num': lambda x: int(x.extract()[1]), 
        # 'rate_details': lambda x: x.extract(), 
    }


    d = {}
    for item, xpath in item2xpath.items():
        # print('\n')
        # print(item)
        selectors = response.xpath(xpath)#.extract()
        values = itemselector2value[item](selectors)
        d[item] = values
        # print(values)
        
        
    return d


d = process_xpath(response)



# All In One

In [ ]:
import json


def process_Json(response):
    xpath = './/script[@type="text/javascript"]//text()'
    selectors = response.xpath(xpath)
    js_data = [i for i in selectors.extract()[3].split('\n')  if '= {' in i]
    js_data = {t[0].strip(): json.loads(t[1][:-1]) for t in [i.split(' = ') for i in js_data]}
    return js_data

def process_Pes(Pes):
    d = {}
    Pes_Model = Pes['model']
    overall = Pes_Model['overall']
    for item in ['responseCount','reviewCount','actualScore','roundedScore',]:
        d[item] = overall[item]
        
    d['score_aggregates'] = Pes_Model["surveyDistribution"]['aggregates']
    d['lastSurveyDate'] = Pes_Model['lastSurveyDate']
    d['cards'] = Pes_Model['cards']
    # at most 20 reviews
    d['reviews'] = Pes['model']['comments']['results']
    return d


def process_viewModel(viewModel):
    cols = ['npi', 'pwid', 'entityType', 'websiteUrl', 'logoUrl', 'imageUrl', 
     'providerUrl', 'displayName', 'displayNamePossessive', 'providerDisplayFullName', 
     'age', 'genderString', 'badges', 'languages', 
     'hasAutoBiography', 'aboutMe', 'aboutMeVideoUrl', 
     'aboutProvider', 'description', 'generatedbiography', 'autoBiography', 'cityNameAndState', 
     'officeLocations', 'officePhone', 'acceptsNewPatients', 
     'availability', 
     'practicingSpecialties', 'medicalSpecialty', 
     'awardsAndRecognitions', 'boardCertifications', 'insuranceAccepted', 
     'isPrimaryLocationMalpracticeCollected', 'malpractices', 
     'sponsorName', 'boardActions', 'memberships', 
     'education', 'hospitals', 'hasConditions', 
     'hasProcedures',  'existingPatientPhone', 
     'readStoryScrolloffset', 'sanctions', 
     'shouldShowVideoContent', 'showPatientVolumeData', 'showVisitingSection', 
     'specialtyHasClinicalFocus', 'specialtyHeaderText', 'suppressCertifications',
     'suppressSurveys', 'syndication', 'testimonies', 'uconnectEnvironment', 'writeMd',
     'conditionsAndProcedures', 'clinicalFocusItems',]
    d = {}
    for i in cols:
        d[i] = viewModel[i]
    return d


def process_xpath(response):
    item2xpath = {
        'biography': './/*[@data-qa-target="premium-biography"]//text()',
        # 'strengths_item': './/li[@class="provider-strengths-item"]',
        # 'rate_score': './/p[@class="score"]//text()',
        # 'rate_num': './/p[@class="survey-count"]//text()',
        # 'rate_details':'.//table[@class="rating-table"]//text()', 
    }
    itemselector2value = {
        'biography': lambda x: x.extract(),
        #  'strengths_item': lambda x: [''.join(i.xpath('.//div//text()').extract()[:4]) for i in x],
        # 'rate_score': lambda x: float(x.extract()[0]), 
        # 'rate_num': lambda x: int(x.extract()[1]), 
        # 'rate_details': lambda x: x.extract(), 
    }
    d = {}
    for item, xpath in item2xpath.items():
        # print('\n')
        # print(item)
        selectors = response.xpath(xpath)#.extract()
        values = itemselector2value[item](selectors)
        d[item] = values
        # print(values)
    return d



def scrapy_healthgraders_physician(url):


    # as we cannot use Download Middleware in Scrapy, requests are used here instead.
    r = requests.get(url, headers = headers)
    # load the text to scrapy-type response
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

    js_data = process_Json(response)
    # [i for i in js_data]
    Pes = js_data['pageState.pes']
    d1 = process_Pes(Pes)

    viewModel = js_data['pageState.viewModel']
    d2 = process_viewModel(viewModel)


    d3 = process_xpath(response)

    d = dict(d1, **d2, **d3)
    return d


def get_healthgraders_reviews(url, reviewCount):

    review_url = 'https://www.healthgrades.com/api4/providerprofile/comments'
    pwid = url.split('-')[-1]
    # pagenum = 1
    perPage = 10
    
    L = []
    
    for pagenum in range(1, int(reviewCount / perPage) + 2):
        forms = {
            'currentPage': str(pagenum), 
            'includeAllAnswers': 'ture',
            'perPage': str(perPage),
            'pwid': pwid.upper(),
            'sortOption': str(1)
        }
        print(forms)

        r = requests.post(review_url, headers = headers, data = forms)

        d = r.json().get('results', [])
        if type(d) != list: d = []
        print(len(d))
        L.append(d)
        
    return sum(L, [])



D = {}


for url in url_list:
    print('\n\n' + url)
    doc_info = scrapy_healthgraders_physician(url)
    print('doctor name is: {}'.format(doc_info['providerDisplayFullName']))

    
    # need to further explore?
    reviewCount = doc_info['reviewCount']
    if reviewCount > len(doc_info['reviews']):
        print('We need to collect more reviews: {} vs {}'.format(len(doc_info['reviews']), reviewCount))
        reviews = get_healthgraders_reviews(url, reviewCount)  
        
        commentIds = [i['commentId'] for i in doc_info['reviews']]
        for ind, commentId in enumerate(commentIds):
            # print(commentId)
            commentId_new = reviews[ind]['commentId']
            # print(b)
            assert commentId == commentId_new

        assert len(reviews) == reviewCount
        doc_info['reviews'] = reviews
        
    print('reivew number: {}'.format(len(doc_info['reviews'])))
    doc_info['url'] = url
    D[url] = doc_info
    
    


In [ ]:
print([i for i in doc_info])